<a href="https://colab.research.google.com/github/KimJae-hee/kyakyakyakya/blob/master/%ED%83%9C%EC%96%91%EA%B4%91_%EB%B0%9C%EC%A0%84_%EC%98%88%EC%B8%A1_%EB%AA%A8%EB%8D%B8_meanEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install category_encoders

In [58]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse
import category_encoders as ce
import sys
from keras.layers.advanced_activations import LeakyReLU, PReLU
np.set_printoptions(threshold=sys.maxsize)
np.random.seed(0)

### 우선 One-Hot Encoding으로 진행하자.

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
dat = pd.read_csv('/content/drive/My Drive/P-SAT/태양광 예측/진짜 데이터 셋/final_data0723_3.csv', encoding = 'CP949')
dat = dat.set_index(pd.to_datetime(dat['date']))
submit_dat = dat[dat['y'].isnull()]
dat_ffnn = dat[dat['y'].notnull()]

In [61]:
dat_ffnn["temp"] /= 100
dat_ffnn["rain"] /= 100
dat_ffnn["wind_speed"] /= 100
dat_ffnn["humidity"] /=100
dat_ffnn["rain_prob"] /=100
dat_ffnn["altitude"] /=10
dat_ffnn["azimuth"] /= 100
dat_ffnn["temp_diff"] /= 100
dat_ffnn["ws_diff"] /=100
dat_ffnn["temp_max"] /= 100
dat_ffnn["hum_max"] /=100
dat_ffnn["ws_max"] /=100
dat_ffnn["temp_mean"] /=100
dat_ffnn["hum_mean"] /= 100
dat_ffnn["ws_mean"] /= 100

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [62]:
dat_ffnn.columns

Index(['y', 'date', 'month', 'day', 'wday', 'hour', 'temp', 'rain',
       'wind_speed', 'humidity', 'rain_prob', 'rain_type', 'skycondition',
       'ymd', 'daily', 'daily_smooth', 'pm10_cat', 'pm2_5_cat', 'altitude',
       'azimuth', 'sin_time', 'cos_time', 'temp_diff', 'hum_diff', 'ws_diff',
       'temp_max', 'hum_max', 'ws_max', 'temp_mean', 'hum_mean', 'ws_mean',
       'rain_sum', 'rain_yn'],
      dtype='object')

In [63]:
drop_columns = ["date", "wday"]

In [68]:
train_1 = dat_ffnn[(dat_ffnn["ymd"] < "2020-01-24")].drop(drop_columns,axis=1)
train_2 = dat_ffnn[(dat_ffnn["ymd"] < "2020-03-24")].drop(drop_columns,axis=1)
train_3 = dat_ffnn[(dat_ffnn["ymd"] < "2020-05-24")].drop(drop_columns,axis=1)

val_1 = dat_ffnn[((dat_ffnn["ymd"] > "2020-01-23") & (dat_ffnn["ymd"] <"2020-01-31"))].drop(drop_columns,axis=1)
val_2 = dat_ffnn[((dat_ffnn["ymd"] > "2020-03-23") & (dat_ffnn["ymd"] <"2020-03-31"))].drop(drop_columns,axis=1)
val_3 = dat_ffnn[((dat_ffnn["ymd"] > "2020-05-23") & (dat_ffnn["ymd"] <"2020-05-31"))].drop(drop_columns,axis=1)

test_1 = submit_dat[(submit_dat["ymd"] == "2020-01-31")].drop(drop_columns,axis=1)
test_2 = submit_dat[(submit_dat["ymd"] == "2020-03-31")].drop(drop_columns,axis=1)
test_3 = submit_dat[(submit_dat["ymd"] == "2020-05-31")].drop(drop_columns,axis=1)

In [69]:
def mean_encoder(train,test,categorical_var):
    ## mean encoding
    cate_var = categorical_var
    train = train.drop(["ymd"],axis = 1)
    test = test.drop(["ymd"],axis = 1)
    for i in cate_var:
        tg_encoder = ce.target_encoder.TargetEncoder(cols=[i])
        tg_encoder.fit(X = train[i],y = train['y'])
        train[i] = tg_encoder.transform(train[i])
        test[i] = tg_encoder.transform(test[i])
        X_train,y_train = train.drop(['y'],axis=1),train.loc[:,['y']]
        X_valid,y_valid = test.drop(['y'],axis=1),test.loc[:,['y']]
    return (X_train,y_train, X_valid, y_valid)

def lossloss(y, y_hat):
  result = []
  for i in range(int(y.shape[0]/24)):
    start = 24*i
    end = (24*(i+1))
    tmp = sum(abs(y[start:end] - y_hat[start:end])*y[start:end]/sum(y[start:end]))/133
    result.append(tmp)
  loss = np.mean(result)
  return loss

def RMSE(y, y_hat):
  result = mse(y, y_hat)**0.5
  return result

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

def modeling(data_set, model, batchsize = 64, learning_rate = 300):
  weighted = []
  rmse = []
  y_hat_result = []
  for i in range(len(data_set)):
    X_train = data_set[i][0]
    Y_train = data_set[i][1]
    X_val = data_set[i][2]
    Y_val = data_set[i][3]

    model.fit(X_train, Y_train, epochs = 300, batch_size = batchsize, validation_split = 0.3, verbose = 0, callbacks = [mc, early_stopping])
    
    y_hat = model.predict(X_val)
    y_hat = y_hat.flatten()
    y_hat_tmp = y_hat

    tmp_weighted = lossloss(Y_val, y_hat)
    tmp_rmse = RMSE(Y_val, y_hat)

    weighted.append(tmp_weighted)
    rmse.append(tmp_rmse)
    y_hat_result = y_hat_tmp
    print(i, "번째 validiaion 결과는 다음과 같습니다.", ":::", "가중평균: ", tmp_weighted, "RMSE: ", tmp_rmse )

  print("   ")
  print(np.mean(weighted))
 # return y_hat_result
  return weighted, y_hat_result

In [70]:
cate_list = ["month", "skycondition", "pm10_cat", "pm2_5_cat", "rain_yn", "hour" ] #인코딩을 위한 변수 

In [71]:
x_train_1, y_train_1, x_val_1, y_val_1 = mean_encoder(train_1, val_1, cate_list)
x_train_2, y_train_2, x_val_2, y_val_2 = mean_encoder(train_2, val_2, cate_list)
x_train_3, y_train_3, x_val_3, y_val_3 = mean_encoder(train_3, val_3, cate_list)

In [72]:
data_set_list = [[x_train_1, y_train_1, x_val_1, y_val_1],
                 [x_train_2, y_train_2, x_val_2, y_val_2],
                 [x_train_3, y_train_3, x_val_3, y_val_3]]

### 모델 정의

In [73]:
dim_x_train_1 = x_train_1.shape

# 1. 모델 구성하기
model = Sequential()
model.__init__
model.add(Dense(40, input_dim= dim_x_train_1[1], activation='sigmoid'))
model.add(Dense(20))
model.add(Dense(1, activation = "linear"))
# 2. 모델 학습과정 설정하기
model.compile(optimizer = "adamax", loss = root_mean_squared_error, 
              metrics =["mse"])

# 3. 모델 학습시키기
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

In [74]:
modeling(data_set_list, model)

KeyboardInterrupt: ignored